# AI Consultant

In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

In [2]:
# Ayt next im going to create a basic instance
consultant_bot = ChatBot(
    "FacultyConsultant",
    storage_adapter="chatterbot.storage.SQLStorageAdapter",
    database_uri="sqlite:///consultant_db.sqlite3"
)

In [3]:
# Here im trying to diversify the training data a bit more
import sqlite3

conn = sqlite3.connect("faculty_evaluations.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG'")
rows = cursor.fetchall()

qa_pairs = []

for name, score, grade in rows:
    responses = [
        f"{name} has a mean score of {score} and a grade of {grade}.",
        f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}.",
        f"The evaluation for {name} shows a mean score of {score} and grade {grade}."
    ]
    
    questions = [
        f"What is the evaluation for {name}?",
        f"Tell me about {name}.",
        f"How did {name} perform?",
        f"Evaluation for {name}?"
    ]
    
    for q in questions:
        for r in responses:
            trainer.train([q, r])

In [4]:
# Just gotta confirm that the database has been created and tables exist
import sqlite3

conn = sqlite3.connect("faculty_evaluations.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

conn.close()



[('faculty_evaluations',)]


In [5]:
# Ok now that ik that lets move on
import sqlite3

conn = sqlite3.connect("faculty_evaluations.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG'")
rows = cursor.fetchall()

qa_pairs = []

for name, score, grade in rows:
    qa_pairs.append([
        f"What is the evaluation for {name}?",
        f"{name} has a mean score of {score} and a grade of {grade}."
    ])

for conversation in qa_pairs:
    trainer.train(conversation)

In [6]:
# Ok lets test it out
response = consultant_bot.get_response("What is the evaluation for Afundi, Patrick Omuhinda?")
print(response)

What is the evaluation for Afundi, Patrick Omuhinda?


ok the bot is alive and is giving out the defualt response so that great.Now the next step is to verify that it’s also pulling from the DB-trained Q&A pairs. Right now, it’s defaulting to the generic greeting because that’s the strongest match in its training set.


In [7]:
response = consultant_bot.get_response("What is the evaluation for Afundi, Patrick Omuhinda?")
print(response)

What is the evaluation for Afundi, Patrick Omuhinda?


ok so I ran that code and got the following responses with each rerun, 'What can you do?','Hello','What is the evaluation for Afundi, Patrick Omuhinda?' just those three on loop
Ok so its just looping thru some basic best match responses. So lets adjust things

so i made a change to the upper section using a threshold and i noticed that if there is a difference in spacing or phrasing or even typing the bot break down and that isnt good so lets fix that

In [8]:
# Here im trying to diversify the training data a bit more
qa_pairs = []

for name, score, grade in rows:
    responses = [
        f"{name} has a mean score of {score} and a grade of {grade}.",
        f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}.",
        f"The evaluation for {name} shows a mean score of {score} and grade {grade}."
    ]
    
    questions = [
        f"What is the evaluation for {name}?",
        f"Tell me about {name}.",
        f"How did {name} perform?",
        f"Evaluation for {name}?"
    ]
    
    for q in questions:
        for r in responses:
            trainer.train([q, r])

In [9]:
response = consultant_bot.get_response("How did Afundi perform?")
print(response)

How did Afundi perform?


so apperently ChatterBot sometimes echoes the input when confidence is low, which explains why we saw “How did Afundi perform?” as a response.


In [10]:
# also i want to adjust the thresholds for similarity matching
consultant_bot = ChatBot(
    "FacultyConsultant",
    storage_adapter="chatterbot.storage.SQLStorageAdapter",
    database_uri="sqlite:///consultant_db.sqlite3",
    logic_adapters=[
        {
            "import_path": "chatterbot.logic.BestMatch",
            "default_response": "I can answer faculty evaluation questions if you ask about a specific instructor.",
            "maximum_similarity_threshold": 0.75
        }
    ]
)

In [11]:
# So lets add more vairaitions for the responses
for name, score, grade in rows:
    questions = [
        f"What is the evaluation for {name}?",
        f"Tell me about {name}.",
        f"How did {name} perform?",
        f"Evaluation for {name}?",
        f"Performance of {name}?"
    ]
    
    responses = [
        f"{name} has a mean score of {score} and a grade of {grade}.",
        f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}.",
        f"The evaluation for {name} shows a mean score of {score} and grade {grade}."
    ]
    
    for q in questions:
        for r in responses:
            trainer.train([q, r])

In [12]:
response = consultant_bot.get_response("How did Afundi perform?")
print(response)

How did Behr perform?


In [13]:
response = consultant_bot.get_response("evaluation for Afundi?")
print(response)

I can answer faculty evaluation questions if you ask about a specific instructor.


bet so atleast it knows that afundi is Afundi patrick omuhinda.
now we’re ready to make this bot truly smart. Instead of relying on memorized Q&A, we’ll wire it to query the SQLite database live when it detects an instructor’s name. That way, even if the phrasing is new or untrained, the bot can still respond accurately.


# Lookup wiring

In [14]:
# first we gotta create the lookup function
import sqlite3
import re

def normalize_name(name):
    name = str(name).lower().strip()
    name = re.sub(r"[^a-z ]", "", name)
    name = re.sub(r"\s+", " ", name)
    return name

def lookup_instructor_evaluation(user_input):
    # Normalize input
    normalized_input = normalize_name(user_input)

    # Connect to DB
    conn = sqlite3.connect("faculty_evaluations.sqlite")
    cursor = conn.cursor()

    # Load instructor names
    cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG'")
    rows = cursor.fetchall()

    # Try to match instructor name
    for name, score, grade in rows:
        if normalize_name(name) in normalized_input:
            return f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}."

    return "I couldn’t find that instructor in the database. Please check the spelling or try a different name."

In [15]:
# now just gotta wrap it ina bot response
user_query = "How did Behr perform?"
response = lookup_instructor_evaluation(user_query)
print(response)

I couldn’t find that instructor in the database. Please check the spelling or try a different name.


In [16]:
user_query = "How did Afundi perform?"
response = lookup_instructor_evaluation(user_query)
print(response)

I couldn’t find that instructor in the database. Please check the spelling or try a different name.


ok this is a bit concerning now. But im thinking, a hybrid chatbot logic so the AI can handle both trained responses and dynamic database lookups. This will make it truly intelligent: if ChatterBot doesn’t know the answer, it’ll fall back to querying the SQLite DB live.


In [17]:
# So lets define the function
def lookup_instructor_evaluation(user_input):
    import sqlite3
    import re

    def normalize_name(name):
        name = str(name).lower().strip()
        name = re.sub(r"[^a-z ]", "", name)
        name = re.sub(r"\s+", " ", name)
        return name

    normalized_input = normalize_name(user_input)

    conn = sqlite3.connect("faculty_evaluations.sqlite")
    cursor = conn.cursor()
    cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG'")
    rows = cursor.fetchall()

    for name, score, grade in rows:
        if normalize_name(name) in normalized_input:
            return f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}."

    return None  # Return None if no match found

In [18]:
# then let me combine the two
def get_consultant_response(user_query):
    # First try ChatterBot
    response = consultant_bot.get_response(user_query)
    confidence = float(response.confidence)

    # If confidence is high, return ChatterBot response
    if confidence >= 0.75:
        return str(response)

    # Otherwise, try dynamic DB lookup
    db_response = lookup_instructor_evaluation(user_query)
    if db_response:
        return db_response

    # Fallback if nothing matches
    return "I couldn’t find that instructor in the database. Please check the spelling or try a different name."

In [19]:
# Now lets test it out
user_query = "How did Akosa perform?"
response = get_consultant_response(user_query)
print(response)

How did Behr perform?


ok so ran the functions and combined and that worked. But im only getting these two responses 'I couldn’t find that instructor in the database. Please check the spelling or try a different name.' and 'How did Afundi perform?'. Also ive noticed if I change the name to like Akosa, it loops thru the responses then it adds in Akosa instead of Afundi after a while

In [20]:
def get_consultant_response(user_query):
    response = consultant_bot.get_response(user_query)
    confidence = float(response.confidence)

    if confidence >= 0.75:
        return str(response)

    db_response = lookup_instructor_evaluation(user_query)
    if db_response:
        return db_response

    return "I couldn’t find that instructor in the database. Please check the spelling or try a different name."

In [21]:
# Ok so im thinking that i can use partial matching to get the rows that matched 
def lookup_instructor_evaluation(user_input):
    import sqlite3
    import re

    def normalize_name(name):
        name = str(name).lower().strip()
        name = re.sub(r"[^a-z ]", "", name)
        name = re.sub(r"\s+", " ", name)
        return name

    normalized_input = normalize_name(user_input)
    input_tokens = set(normalized_input.split())

    conn = sqlite3.connect("faculty_evaluations.sqlite")
    cursor = conn.cursor()
    cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG'")
    rows = cursor.fetchall()

    for name, score, grade in rows:
        normalized_name = normalize_name(name)
        name_tokens = set(normalized_name.split())

        # Match if there's any token overlap
        if input_tokens & name_tokens:
            return f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}."

    return None

In [22]:
# now lets test it again
user_query = "How did Akosa perform?"
response = get_consultant_response(user_query)
print(response)

How did Behr perform?


ayt that looping is an issue so lets try some fuzzy matching like we did in the database

In [23]:
from chatterbot.trainers import ChatterBotCorpusTrainer

# Create a new trainer for the chatbot
trainer = ChatterBotCorpusTrainer(consultant_bot)

# Train based on the english corpus
trainer.train("chatterbot.corpus.english")

# Train based on english greetings corpus
trainer.train("chatterbot.corpus.english.greetings")

# Train based on the english conversations corpus
trainer.train("chatterbot.corpus.english.conversations")

Training corpus: 19it [00:02,  6.39it/s]
Training corpus: 1it [00:00, 26.14it/s]
Training corpus: 1it [00:00,  8.92it/s]


this code chunk above was just to make sure teh bot understands english and basic greetings

In [24]:
# So id decided to update the fuzzy matching logic and to see whats going on with it
from rapidfuzz import fuzz
import sqlite3
import re

def normalize_name(name):
    name = str(name).lower().strip()
    name = re.sub(r"[^a-z ]", "", name)
    name = re.sub(r"\s+", " ", name)
    return name

def lookup_instructor_evaluation(user_input, threshold=70):
    normalized_input = normalize_name(user_input)

    conn = sqlite3.connect("faculty_evaluations.sqlite")
    cursor = conn.cursor()
    cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG'")
    rows = cursor.fetchall()

    best_match = None
    best_score = 0

    for name, score, grade in rows:
        normalized_name = normalize_name(name)
        similarity = fuzz.partial_ratio(normalized_input, normalized_name)

        if similarity > best_score:
            best_score = similarity
            best_match = (name, score, grade)

    if best_match and best_score >= threshold:
        name, score, grade = best_match
        return f"Based on evaluations, {name} received a grade of {grade} with a mean score of {score}."

    return None

In [25]:
def get_consultant_response(user_query):
    response = consultant_bot.get_response(user_query)
    confidence = float(response.confidence)

    if confidence >= 0.75:
        return str(response)

    db_response = lookup_instructor_evaluation(user_query)
    if db_response:
        return db_response

    return "I couldn’t find that instructor in the database. Please check the spelling or try a different name."

In [26]:
# now lets test it again
user_query = "can you give me a type of question that i can ask you"
response = get_consultant_response(user_query)
print(response)

I couldn’t find that instructor in the database. Please check the spelling or try a different name.


ok so it can handle basic conversations but i want more. I need it to understand intent. So lets build one

In [27]:
def parse_intent(user_input):
    import re

    input_lower = user_input.lower()

    # Instructor lookup
    if "how did" in input_lower or "tell me about" in input_lower:
        return {"intent": "lookup", "name": user_input}

    # Score threshold
    match_score = re.search(r"score[s]? above (\d+(\.\d+)?)", input_lower)
    if match_score:
        return {"intent": "score_filter", "threshold": float(match_score.group(1))}

    # Grade filter
    match_grade = re.search(r"grade[s]? (of )?([abc])", input_lower)
    if match_grade:
        return {"intent": "grade_filter", "grade": match_grade.group(2).upper()}

    # Top performers
    match_top = re.search(r"top (\d+)", input_lower)
    if match_top:
        return {"intent": "top_ranked", "count": int(match_top.group(1))}

    # Negative feedback
    if "negative feedback" in input_lower or "bad comments" in input_lower:
        return {"intent": "negative_feedback"}

    # Program filter
    match_program = re.search(r"(undergraduate|doctoral|graduate)", input_lower)
    if match_program:
        return {"intent": "program_filter", "program": match_program.group(1).upper()}

    return {"intent": "unknown"}

In [28]:
# now lets test it again
user_query = "How did Afundi Patrick perform"
response = get_consultant_response(user_query)
print(response)

How did Afundi perform?


In [29]:
# Ayt let me add in a score threshold for the data
def parse_intent(user_input):
    import re
    input_lower = user_input.lower()

    # Score threshold intent
    match_score = re.search(r"score[s]? above (\d+(\.\d+)?)", input_lower)
    if match_score:
        return {"intent": "score_filter", "threshold": float(match_score.group(1))}

    # Top performers intent
    match_top = re.search(r"top (\d+)", input_lower)
    if match_top:
        return {"intent": "top_ranked", "count": int(match_top.group(1))}

    # Instructor lookup (fallback)
    if "how did" in input_lower or "tell me about" in input_lower or "evaluation for" in input_lower:
        return {"intent": "lookup", "name": user_input}

    return {"intent": "unknown"}

In [30]:
# Next i need some handlers
import sqlite3

def handle_score_filter(threshold):
    conn = sqlite3.connect("faculty_evaluations.sqlite")
    cursor = conn.cursor()
    cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG' AND mean_score > ?", (threshold,))
    rows = cursor.fetchall()

    if rows:
        response = f"Instructors with scores above {threshold}:\n"
        for name, score, grade in rows:
            response += f"- {name}: {score} ({grade})\n"
        return response.strip()
    return f"No instructors scored above {threshold}."

def handle_top_ranked(count):
    conn = sqlite3.connect("faculty_evaluations.sqlite")
    cursor = conn.cursor()
    cursor.execute("SELECT full_name, mean_score, letter_grade FROM faculty_evaluations WHERE program='UNDG' ORDER BY mean_score DESC LIMIT ?", (count,))
    rows = cursor.fetchall()

    if rows:
        response = f"Top {count} performers:\n"
        for name, score, grade in rows:
            response += f"- {name}: {score} ({grade})\n"
        return response.strip()
    return f"No data available for top {count} performers."

In [31]:
# Then i gotta combine everything
def get_consultant_response(user_query):
    # First parse intent
    intent = parse_intent(user_query)

    if intent["intent"] == "score_filter":
        return handle_score_filter(intent["threshold"])

    elif intent["intent"] == "top_ranked":
        return handle_top_ranked(intent["count"])

    elif intent["intent"] == "lookup":
        db_response = lookup_instructor_evaluation(user_query)
        if db_response:
            return db_response
        return "I couldn’t find that instructor in the database."

    else:
        # Fall back to ChatterBot
        response = consultant_bot.get_response(user_query)
        return str(response)

In [43]:
# Now we test it 
# now lets test it again
user_query = "Who scored above 4.5?"
response = get_consultant_response(user_query)
print(response)

# Now we test it 
# now lets test it again
user_query = "what annoys you?"
response = get_consultant_response(user_query)
print(response)

What is the evaluation for Afundi, Patrick Omuhinda?
A lot of things, like all the other digits other than 0 and 1.


In [33]:
import re

def parse_intent(user_input):
    text = user_input.lower().strip()

    # Top N performers (supports "top 5", "top five")
    m_top = re.search(r"top\s+(\d+|one|two|three|four|five|six|seven|eight|nine|ten)", text)
    if m_top:
        word = m_top.group(1)
        word_to_num = {
            "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
            "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10
        }
        count = int(word) if word.isdigit() else word_to_num.get(word, 5)
        return {"intent": "top_ranked", "count": count}

    # Score filters: above/below/exact
    m_above = re.search(r"(score|scores|mean score)\s*(above|over|greater than)\s*(\d+(\.\d+)?)", text)
    if m_above:
        return {"intent": "score_filter", "op": ">", "threshold": float(m_above.group(3))}

    m_below = re.search(r"(score|scores|mean score)\s*(below|under|less than)\s*(\d+(\.\d+)?)", text)
    if m_below:
        return {"intent": "score_filter", "op": "<", "threshold": float(m_below.group(3))}

    m_equal = re.search(r"(score|scores|mean score)\s*(equal|equals|=|is)\s*(\d+(\.\d+)?)", text)
    if m_equal or re.search(r"who scored\s+(\d+(\.\d+)?)", text):
        val = float(m_equal.group(3)) if m_equal else float(re.search(r"who scored\s+(\d+(\.\d+)?)", text).group(1))
        return {"intent": "score_filter", "op": "=", "threshold": val}

    # Grade filters: "who got an A", "grade B"
    m_grade = re.search(r"(grade|got a|received a)\s*([abc])\b", text)
    if m_grade:
        return {"intent": "grade_filter", "grade": m_grade.group(2).upper()}

    # Instructor lookup
    if any(kw in text for kw in ["how did", "tell me about", "evaluation for", "performance of", "results for"]):
        return {"intent": "lookup", "name": user_input}

    return {"intent": "unknown"}

In [34]:
def get_consultant_response(user_query):
    intent = parse_intent(user_query)

    # Intent-first routing
    if intent["intent"] == "top_ranked":
        return handle_top_ranked(intent["count"])

    if intent["intent"] == "score_filter":
        return handle_score_filter(intent["op"], intent["threshold"])

    if intent["intent"] == "grade_filter":
        return handle_grade_filter(intent["grade"])

    if intent["intent"] == "lookup":
        db_resp = lookup_instructor_evaluation(user_query)
        if db_resp:
            return db_resp
        return "I couldn’t find that instructor in the database."

    # Last resort: ChatterBot
    response = consultant_bot.get_response(user_query)
    return str(response)

In [35]:
import sqlite3

def debug_db_overview():
    conn = sqlite3.connect("faculty_evaluations.sqlite")
    c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM faculty_evaluations")
    total = c.fetchone()[0]

    c.execute("SELECT DISTINCT program FROM faculty_evaluations ORDER BY program")
    programs = [row[0] for row in c.fetchall()]

    c.execute("SELECT full_name, mean_score, letter_grade, program FROM faculty_evaluations LIMIT 5")
    sample = c.fetchall()
    conn.close()

    return {"total_rows": total, "programs": programs, "sample": sample}

info = debug_db_overview()
print("Total rows:", info["total_rows"])
print("Programs:", info["programs"])
print("Sample:", info["sample"])

Total rows: 0
Programs: []
Sample: []


this is it, the bane of my existence, there was nothing to be found. Thats why i was gettinf nothing

In [40]:
from chatterbot import ChatBot


bot = ChatBot(
    'Math & Time Bot',
    logic_adapters=[
        'chatterbot.logic.MathematicalEvaluation',
        'chatterbot.logic.TimeLogicAdapter'
    ]
)

# Print an example of getting one math based response
response = bot.get_response('Who is john f. kennedy')
print(response)


response = bot.get_response('What is baseball')
print(response)



The current time is 05:19 PM
The current time is 05:19 PM


ok so I decided to go back to my folder and check on the db. And I found 5 dbs are present . So I went thru each. And this is what I found
consultant db has three tables statement: text search_text conversation created_at in_response_to search_in_response_to persona id
tag : name id
tag associations: tag_id statement_id
from what I can gather it seems to be a knowledge base for the defuatl db that can answer simple questions such as what is baseball or soccer. and when I ased our consultant Ai what is baseball it told me 'a game with tall players' and another question I gave it was tell me a joke, and it told me 'Did you hear the one about the mountain goats in the andes? It was "ba a a a a a d'
so it seems we were connecting to this database instead of the one that we made
I still ran another question from the db that asked what annoys you and I got 'A lot of things, like all the other digits other than 0 and 1.'
so that means our bot was connected to a db and functions as should, but It was connected to the wrong db. But this is atleast good cause the plane was to move from a general Chat bot to
db .sqlite4 is where the past questions are stored it has these 3 tables statement: each row is: text search_text conversation created_at in_response_to search_in_response_to persona id
tag: name id
tag_association: tag_id statement_id
faculty_evaluation.db is the db that we made it has the tables that we made for it Courses Deans Evaluations Faculty FeedbackCorpus Sections TeachingAssingments sqlite_sequence: name seq
faculty_evaluation.sqlite is entirely empty with no tables in it either that's why its 0kbs
faculty_evaluations.sqlite has this table faculty_evalations: full_name mean_score letter_grade program So it seems we were connecting to the worng db. So what im thinking now is that w'll scrap everything that we did before and start afresh with this new knowledge. Cause from what weve learnt is that we did correctly build a generic chatbot and it operated as intedended, we were just asking it the wrong questions and expecting it to answer what it doesn't know. So im going to create a new notebook and we''ll start a fresh
